## Import Packages and Data

In [2]:
#Import Package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import mean_squared_log_error
from sklearn import preprocessing
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance

In [13]:
#Import datasets
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
data_holidays = pd.read_csv('holidays_events.csv')
data_oil = pd.read_csv('oil.csv')
data_stores = pd.read_csv('stores.csv')

## Check whether there are any NaN in the datasets

In [14]:
data_train.isnull().any()

id             False
date           False
store_nbr      False
family         False
sales          False
onpromotion    False
dtype: bool

In [5]:
data_test.isnull().any()

id             False
date           False
store_nbr      False
family         False
onpromotion    False
dtype: bool

In [6]:
data_holidays.isnull().any()

date           False
type           False
locale         False
locale_name    False
description    False
transferred    False
dtype: bool

In [7]:
data_oil.isnull().any()

date          False
dcoilwtico     True
dtype: bool

In [17]:
#Fill the NaN with lastest price
data_oil_new = pd.DataFrame()
data_oil_new = data_oil
for i in range(len(data_oil_new)):
    if data_oil_new.iloc[i][1] > 0:
        pass
    elif i != 0:
        data_oil_new['dcoilwtico'][i] = data_oil_new['dcoilwtico'][i-1]
data_oil_new['dcoilwtico'][0] = data_oil_new['dcoilwtico'][1]

C:\Users\user\AppData\Local\Temp/ipykernel_13504/3181917605.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_oil_new['dcoilwtico'][i] = data_oil_new['dcoilwtico'][i-1]
C:\Users\user\AppData\Local\Temp/ipykernel_13504/3181917605.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_oil_new['dcoilwtico'][0] = data_oil_new['dcoilwtico'][1]


In [18]:
data_oil_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1218 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [14]:
data_stores.isnull().any()

store_nbr    False
city         False
state        False
type         False
cluster      False
dtype: bool

In [48]:
data_holidays["locale"].value_counts()

National    174
Local       152
Regional     24
Name: locale, dtype: int64

## Merging Data into a Table

In [19]:
df_train = data_train
df_train = df_train.merge(data_oil_new, how='left', on = 'date')
data_stores['country'] = 'Ecuador' 
df_train = df_train.merge(data_stores, how='left', on = 'store_nbr')

In [20]:
df_test = data_test
df_test = df_test.merge(data_oil_new, how='left', on = 'date')
df_test = df_test.merge(data_stores, how='left', on = 'store_nbr')

In [21]:
data_holidays_country = data_holidays.rename({"locale_name":"country"}, axis='columns')
df_train = df_train.merge(data_holidays_country, how='left', on = ['date', 'country'] )

In [22]:
df_test = df_test.merge(data_holidays_country, how='left', on = ['date', 'country'] )

## Transforming Data into the Right Format

In [24]:
df_train = df_train.drop(['country', 'type_y', 'description','transferred'], axis=1)
df_train['national_holiday'] = pd.get_dummies(df_train['locale'])
df_train = df_train.drop(['locale'], axis=1)

In [26]:
df_test = df_test.drop(['country', 'type_y', 'description','transferred'], axis=1)
df_test['national_holiday'] = 0
df_test = df_test.drop(['locale'], axis=1)

In [27]:
df_train = df_train.rename({"type_x" : "store_type"}, axis = 'columns')
df_test = df_test.rename({"type_x" : "store_type"}, axis = 'columns')

In [31]:
data_holidays_region = data_holidays.rename({"locale_name":"state"}, axis='columns')
df_train = df_train.merge(data_holidays_region, how='left', on = ['date', 'state'] )

In [33]:
df_test = df_test.merge(data_holidays_region, how='left', on = ['date', 'state'] )

In [34]:
df_train = df_train.drop(['type', 'description','transferred'], axis=1)
df_train['region_holiday'] = df_train['locale']=='Regional'
df_train = df_train.drop(['locale'], axis=1)

In [35]:
df_test = df_test.drop(['type', 'description','transferred'], axis=1)
df_test['region_holiday'] = df_test['locale']=='Regional'
df_test = df_test.drop(['locale'], axis=1)

In [36]:
data_holidays_city = data_holidays.rename({"locale_name":"city"}, axis='columns')
df_train = df_train.merge(data_holidays_city, how='left', on = ['date', 'city'] )
df_test = df_test.merge(data_holidays_city, how='left', on = ['date', 'city'] )

In [37]:
df_train = df_train.drop(['type', 'description','transferred'], axis=1)
df_train['city_holiday'] = df_train['locale']=='Local'
df_train = df_train.drop(['locale'], axis=1)

In [38]:
df_test = df_test.drop(['type', 'description','transferred'], axis=1)
df_test['city_holiday'] = df_test['locale']=='Local'
df_test = df_test.drop(['locale'], axis=1)

In [39]:
df_train = df_train.replace(to_replace = False, value =0)
df_train = df_train.replace(to_replace = True, value =1)

In [40]:
df_test = df_test.replace(to_replace = False, value =0)
df_test = df_test.replace(to_replace = True, value =1)

In [31]:
#df_train['holidays'] = (df_train["national_holiday"]>0) | (df_train["region_holiday"]>0) | (df_train["city_holiday"]>0)
#df_test['holidays'] = (df_test["national_holiday"]>0) | (df_test["region_holiday"]>0) | (df_test["city_holiday"]>0)

In [32]:
#df_train = df_train.replace(to_replace = False, value =0)
#df_train = df_train.replace(to_replace = True, value =1)
#df_test = df_test.replace(to_replace = False, value =0)
#df_test = df_test.replace(to_replace = True, value =1)

In [49]:
df_train

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,city,state,store_type,cluster,national_holiday,region_holiday,city_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,93.14,Quito,Pichincha,D,13,1,0,0
1,1,2013-01-01,1,BABY CARE,0.000,0,93.14,Quito,Pichincha,D,13,1,0,0
2,2,2013-01-01,1,BEAUTY,0.000,0,93.14,Quito,Pichincha,D,13,1,0,0
3,3,2013-01-01,1,BEVERAGES,0.000,0,93.14,Quito,Pichincha,D,13,1,0,0
4,4,2013-01-01,1,BOOKS,0.000,0,93.14,Quito,Pichincha,D,13,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3008275,3000883,2017-08-15,9,POULTRY,438.133,0,47.57,Quito,Pichincha,B,6,0,0,0
3008276,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,47.57,Quito,Pichincha,B,6,0,0,0
3008277,3000885,2017-08-15,9,PRODUCE,2419.729,148,47.57,Quito,Pichincha,B,6,0,0,0
3008278,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,47.57,Quito,Pichincha,B,6,0,0,0


Focus on data after 2017-01-01

In [50]:
df_train = df_train[df_train['date']>"2016-12-31"]

In [51]:
#Create month, day columns for train set
month_train=[]
day_train=[]
for i in range(len(df_train)):
    month_train.append(df_train.iloc[i][1][5:7])
    day_train.append(df_train.iloc[i][1][8:10])
df_train['month'] = month_train
df_train['day'] = day_train

C:\Users\user\AppData\Local\Temp/ipykernel_13504/1865844059.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['month'] = month_train
C:\Users\user\AppData\Local\Temp/ipykernel_13504/1865844059.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['day'] = day_train


In [52]:
#Create month, day columns for test set
month_test=[]
day_test=[]
for i in range(len(df_test)):
    month_test.append(df_test.iloc[i][1][5:7])
    day_test.append(df_test.iloc[i][1][8:10])
df_test['month'] = month_test
df_test['day'] = day_test

In [62]:
df_train

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,city,state,store_type,cluster,national_holiday,region_holiday,city_holiday,month,day
2603766,2596374,2017-01-01,1,AUTOMOTIVE,0.000,0,NaN,Quito,Pichincha,D,13,1,0,0,01,01
2603767,2596375,2017-01-01,1,BABY CARE,0.000,0,NaN,Quito,Pichincha,D,13,1,0,0,01,01
2603768,2596376,2017-01-01,1,BEAUTY,0.000,0,NaN,Quito,Pichincha,D,13,1,0,0,01,01
2603769,2596377,2017-01-01,1,BEVERAGES,0.000,0,NaN,Quito,Pichincha,D,13,1,0,0,01,01
2603770,2596378,2017-01-01,1,BOOKS,0.000,0,NaN,Quito,Pichincha,D,13,1,0,0,01,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3008275,3000883,2017-08-15,9,POULTRY,438.133,0,47.57,Quito,Pichincha,B,6,0,0,0,08,15
3008276,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,47.57,Quito,Pichincha,B,6,0,0,0,08,15
3008277,3000885,2017-08-15,9,PRODUCE,2419.729,148,47.57,Quito,Pichincha,B,6,0,0,0,08,15
3008278,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,47.57,Quito,Pichincha,B,6,0,0,0,08,15


In [38]:
df_test

,id,date,store_nbr,family,onpromotion,dcoilwtico,city,state,store_type,cluster,national_holiday,region_holiday,city_holiday,month,day
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.80,Quito,Pichincha,D,13,0,0,0,08,16
1,3000889,2017-08-16,1,BABY CARE,0,46.80,Quito,Pichincha,D,13,0,0,0,08,16
2,3000890,2017-08-16,1,BEAUTY,2,46.80,Quito,Pichincha,D,13,0,0,0,08,16
3,3000891,2017-08-16,1,BEVERAGES,20,46.80,Quito,Pichincha,D,13,0,0,0,08,16
4,3000892,2017-08-16,1,BOOKS,0,46.80,Quito,Pichincha,D,13,0,0,0,08,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,47.26,Quito,Pichincha,B,6,0,0,0,08,31
28508,3029396,2017-08-31,9,PREPARED FOODS,0,47.26,Quito,Pichincha,B,6,0,0,0,08,31
28509,3029397,2017-08-31,9,PRODUCE,1,47.26,Quito,Pichincha,B,6,0,0,0,08,31
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,47.26,Quito,Pichincha,B,6,0,0,0,08,31


In [55]:
#Split train set into train set and validation set
data_train = df_train[df_train['date']<="2017-07-15"]
data_val = df_train[df_train['date']>"2017-07-15"]
data_test = df_test

## Creating an Additional Column - Weekday

In [56]:
#Create weekday column for train set
train_date = []
for j in range(len(data_train)):
    train_date.append(datetime.strptime(data_train.iloc[j][1],"%Y-%m-%d"))
s_train_date = pd.Series(train_date)
s_train_date = s_train_date.dt.weekday
s_train_date = s_train_date.tolist()
data_train['weekday'] = s_train_date

C:\Users\user\AppData\Local\Temp/ipykernel_13504/2011225616.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['weekday'] = s_train_date


In [57]:
#Create weekday column for validation set
val_date = []
for j in range(len(data_val)):
    val_date.append(datetime.strptime(data_val.iloc[j][1],"%Y-%m-%d"))
s_val_date = pd.Series(val_date)
s_val_date = s_val_date.dt.weekday
s_val_date = s_val_date.tolist()
data_val['weekday'] = s_val_date

C:\Users\user\AppData\Local\Temp/ipykernel_13504/236884441.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_val['weekday'] = s_val_date


In [58]:
#Create weekday column for test set
test_date = []
for j in range(len(data_test)):
    test_date.append(datetime.strptime(data_test.iloc[j][1],"%Y-%m-%d"))
s_test_date = pd.Series(test_date)
s_test_date = s_test_date.dt.weekday
s_test_date = s_test_date.tolist()
data_test['weekday'] = s_test_date

In [63]:
#Drop duplicate columns
data_train = data_train.drop(['id'], axis=1)
data_val = data_val.drop(['id'], axis=1)
data_test = data_test.drop(['id'], axis=1)

In [ ]:
X_train = data_train.drop(['sales'],axis=1)
y_train = data_train['sales']
X_val = data_val.drop(['sales'],axis=1)
y_val = data_val['sales']

In [65]:
X_train.isnull().any()

date                False
store_nbr           False
family              False
onpromotion         False
dcoilwtico           True
city                False
state               False
store_type          False
cluster             False
national_holiday    False
region_holiday      False
city_holiday        False
month               False
day                 False
weekday             False
dtype: bool

## Cleaining Oil Data

In [66]:
#Use average oil price to fill oil na in train set
train_oil_new = pd.DataFrame()
train_oil_new['date'] = data_train['date']
train_oil_new = train_oil_new.merge(data_oil_new, how='left', on = 'date')
train_oil_new['month'] = train_oil_new['date'].map(lambda x: int(x.replace('-', '')[:6]))
train_oil_new['month_avg'] = train_oil_new.groupby('month')['dcoilwtico'].transform('mean')
train_oil_new['tmp'] = train_oil_new['dcoilwtico'].map(np.isnan)
train_oil_new['month_avg'] = train_oil_new['tmp'] * train_oil_new['month_avg']
train_oil_new['dcoilwtico'].fillna(0, inplace=True)
train_oil_new['oil'] = train_oil_new['dcoilwtico'] + train_oil_new['month_avg']
train_oil_new = train_oil_new.drop(['month', 'month_avg', 'tmp','dcoilwtico'], axis=1)
train_oil = train_oil_new['oil'].tolist()
X_train['oil'] = train_oil

In [67]:
#Use average oil price to fill oil na in validation set
val_oil_new = pd.DataFrame()
val_oil_new['date'] = data_val['date']
val_oil_new = val_oil_new.merge(data_oil_new, how='left', on = 'date')
val_oil_new['month'] = val_oil_new['date'].map(lambda x: int(x.replace('-', '')[:6]))
val_oil_new['month_avg'] = val_oil_new.groupby('month')['dcoilwtico'].transform('mean')
val_oil_new['tmp'] = val_oil_new['dcoilwtico'].map(np.isnan)
val_oil_new['month_avg'] = val_oil_new['tmp'] * val_oil_new['month_avg']
val_oil_new['dcoilwtico'].fillna(0, inplace=True)
val_oil_new['oil'] = val_oil_new['dcoilwtico'] + val_oil_new['month_avg']
val_oil_new = val_oil_new.drop(['month', 'month_avg', 'tmp','dcoilwtico'], axis=1)
val_oil = val_oil_new['oil'].tolist()
X_val['oil'] = val_oil

In [68]:
#Use average oil price to fill oil na in test set
test_oil_new = pd.DataFrame()
test_oil_new['date'] = data_test['date']
test_oil_new = test_oil_new.merge(data_oil_new, how='left', on = 'date')
test_oil_new['month'] = test_oil_new['date'].map(lambda x: int(x.replace('-', '')[:6]))
test_oil_new['month_avg'] = test_oil_new.groupby('month')['dcoilwtico'].transform('mean')
test_oil_new['tmp'] = test_oil_new['dcoilwtico'].map(np.isnan)
test_oil_new['month_avg'] = test_oil_new['tmp'] * test_oil_new['month_avg']
test_oil_new['dcoilwtico'].fillna(0, inplace=True)
test_oil_new['oil'] = test_oil_new['dcoilwtico'] + test_oil_new['month_avg']
test_oil_new = test_oil_new.drop(['month', 'month_avg', 'tmp','dcoilwtico'], axis=1)
test_oil = test_oil_new['oil'].tolist()
data_test['oil'] = test_oil

In [69]:
X_train = X_train.drop(['dcoilwtico', 'date'], axis=1)
X_val = X_val.drop(['dcoilwtico', 'date'], axis=1)
data_test = data_test.drop(['dcoilwtico', 'date'], axis=1)

## Transforming Categorical Data

In [70]:
#Apply labelencoder to deal with the categorize factors
cat_features = ['family', 'store_nbr', 'city', 'state', 'store_type', 'cluster']
for col in cat_features:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_val[col] = le.fit_transform(X_val[col])
    data_test[col] = le.transform(data_test[col])

In [71]:
train_test_data = pd.concat([X_train,X_val])
train_test_data = pd.concat([train_test_data,data_test])

In [72]:
train_test_data

,store_nbr,family,onpromotion,city,state,store_type,cluster,national_holiday,region_holiday,city_holiday,month,day,weekday,oil
2603766,0,0,0,18,12,3,12,1,0,0,01,01,6,52.554091
2603767,0,1,0,18,12,3,12,1,0,0,01,01,6,52.554091
2603768,0,2,0,18,12,3,12,1,0,0,01,01,6,52.554091
2603769,0,3,0,18,12,3,12,1,0,0,01,01,6,52.554091
2603770,0,4,0,18,12,3,12,1,0,0,01,01,6,52.554091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,8,28,1,18,12,1,5,0,0,0,08,31,3,47.260000
28508,8,29,0,18,12,1,5,0,0,0,08,31,3,47.260000
28509,8,30,1,18,12,1,5,0,0,0,08,31,3,47.260000
28510,8,31,9,18,12,1,5,0,0,0,08,31,3,47.260000


In [73]:
train_test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 433026 entries, 2603766 to 28511
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   store_nbr         433026 non-null  int64  
 1   family            433026 non-null  int32  
 2   onpromotion       433026 non-null  int64  
 3   city              433026 non-null  int32  
 4   state             433026 non-null  int32  
 5   store_type        433026 non-null  int32  
 6   cluster           433026 non-null  int64  
 7   national_holiday  433026 non-null  int64  
 8   region_holiday    433026 non-null  int64  
 9   city_holiday      433026 non-null  int64  
 10  month             433026 non-null  object 
 11  day               433026 non-null  object 
 12  weekday           433026 non-null  int64  
 13  oil               433026 non-null  float64
dtypes: float64(1), int32(4), int64(7), object(2)
memory usage: 42.9+ MB


## Transforming Date Data into Dummies Variables

In [74]:
#Create weekday, month, day dummy variables
weekday_dummy = pd.get_dummies(train_test_data['weekday'], prefix ='weekday')
month_dummy = pd.get_dummies(train_test_data['month'], prefix ='month')
day_dummy = pd.get_dummies(train_test_data['day'], prefix ='day')

In [75]:
train_test_data = pd.concat([train_test_data,weekday_dummy], axis =1)
train_test_data = pd.concat([train_test_data,month_dummy], axis =1)
train_test_data = pd.concat([train_test_data,day_dummy], axis =1)
train_test_data = train_test_data.drop(['month', 'day','weekday'], axis=1)

In [77]:
train_test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 433026 entries, 2603766 to 28511
Data columns (total 57 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   store_nbr         433026 non-null  int64  
 1   family            433026 non-null  int32  
 2   onpromotion       433026 non-null  int64  
 3   city              433026 non-null  int32  
 4   state             433026 non-null  int32  
 5   store_type        433026 non-null  int32  
 6   cluster           433026 non-null  int64  
 7   national_holiday  433026 non-null  int64  
 8   region_holiday    433026 non-null  int64  
 9   city_holiday      433026 non-null  int64  
 10  oil               433026 non-null  float64
 11  weekday_0         433026 non-null  uint8  
 12  weekday_1         433026 non-null  uint8  
 13  weekday_2         433026 non-null  uint8  
 14  weekday_3         433026 non-null  uint8  
 15  weekday_4         433026 non-null  uint8  
 16  weekday_5      

In [78]:
train_test_data

,store_nbr,family,onpromotion,city,state,store_type,cluster,national_holiday,region_holiday,city_holiday,...,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
2603766,0,0,0,18,12,3,12,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2603767,0,1,0,18,12,3,12,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2603768,0,2,0,18,12,3,12,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2603769,0,3,0,18,12,3,12,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2603770,0,4,0,18,12,3,12,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,8,28,1,18,12,1,5,0,0,0,...,0,0,0,0,0,0,0,0,0,1
28508,8,29,0,18,12,1,5,0,0,0,...,0,0,0,0,0,0,0,0,0,1
28509,8,30,1,18,12,1,5,0,0,0,...,0,0,0,0,0,0,0,0,0,1
28510,8,31,9,18,12,1,5,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [56]:
#Standardize the oil price 
ooil = train_test_data['oil'].tolist()
mean_oil = np.mean(ooil)
std_oil = np.std(ooil)
for i in range(len(ooil)):
    ooil[i] = (ooil[i]-mean_oil)/std_oil
train_test_data['oil'] = ooil

In [57]:
#scaler = MinMaxScaler()
#scaler.fit(train_test_data)
#train_test_data['oil'] = scaler.transform(train_test_data['oil'])

In [79]:
#Split dataset into train, validation, and test sets
X_train = train_test_data[:len(X_train)]
X_val = train_test_data[len(X_train):len(X_train)+len(X_val)]
X_test = train_test_data[len(X_train)+len(X_val):]

In [80]:
#Apply log to transfer the sales
y_train = np.log(y_train+1) 
y_val = np.log(y_val+1)

## Building model

In [81]:
#Model XGBOOST
xgbr = xgb.XGBRegressor(max_depth =11, n_estimators =20000, learning_rate =0.01, subsample =0.99, reg_alpha =10, reg_lambda=0.2, colsample_bytree=0.8)
xgbr.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_val,y_val)], eval_metric = 'rmse')

[0]	validation_0-rmse:3.95409	validation_1-rmse:3.97687
[1]	validation_0-rmse:3.92016	validation_1-rmse:3.94361
[2]	validation_0-rmse:3.88223	validation_1-rmse:3.90562
[3]	validation_0-rmse:3.84468	validation_1-rmse:3.86819
[4]	validation_0-rmse:3.81184	validation_1-rmse:3.83609
[5]	validation_0-rmse:3.77495	validation_1-rmse:3.79922
[6]	validation_0-rmse:3.73846	validation_1-rmse:3.76274
[7]	validation_0-rmse:3.70254	validation_1-rmse:3.72698
[8]	validation_0-rmse:3.67103	validation_1-rmse:3.69615
[9]	validation_0-rmse:3.63987	validation_1-rmse:3.66572
[10]	validation_0-rmse:3.60480	validation_1-rmse:3.63062
[11]	validation_0-rmse:3.57431	validation_1-rmse:3.60081
[12]	validation_0-rmse:3.53994	validation_1-rmse:3.56636
[13]	validation_0-rmse:3.50586	validation_1-rmse:3.53230
[14]	validation_0-rmse:3.48450	validation_1-rmse:3.51074
[15]	validation_0-rmse:3.45526	validation_1-rmse:3.48216
[16]	validation_0-rmse:3.42203	validation_1-rmse:3.44880
[17]	validation_0-rmse:3.39338	validation

[144]	validation_0-rmse:1.14204	validation_1-rmse:1.17157
[145]	validation_0-rmse:1.13324	validation_1-rmse:1.16275
[146]	validation_0-rmse:1.12620	validation_1-rmse:1.15596
[147]	validation_0-rmse:1.11766	validation_1-rmse:1.14736
[148]	validation_0-rmse:1.10908	validation_1-rmse:1.13879
[149]	validation_0-rmse:1.10239	validation_1-rmse:1.13234
[150]	validation_0-rmse:1.09397	validation_1-rmse:1.12390
[151]	validation_0-rmse:1.08579	validation_1-rmse:1.11574
[152]	validation_0-rmse:1.07762	validation_1-rmse:1.10770
[153]	validation_0-rmse:1.07113	validation_1-rmse:1.10148
[154]	validation_0-rmse:1.06319	validation_1-rmse:1.09344
[155]	validation_0-rmse:1.05520	validation_1-rmse:1.08539
[156]	validation_0-rmse:1.04908	validation_1-rmse:1.07951
[157]	validation_0-rmse:1.04131	validation_1-rmse:1.07170
[158]	validation_0-rmse:1.03527	validation_1-rmse:1.06594
[159]	validation_0-rmse:1.02776	validation_1-rmse:1.05843
[160]	validation_0-rmse:1.02036	validation_1-rmse:1.05107
[161]	validati

[286]	validation_0-rmse:0.56782	validation_1-rmse:0.60565
[287]	validation_0-rmse:0.56685	validation_1-rmse:0.60483
[288]	validation_0-rmse:0.56525	validation_1-rmse:0.60334
[289]	validation_0-rmse:0.56374	validation_1-rmse:0.60188
[290]	validation_0-rmse:0.56219	validation_1-rmse:0.60040
[291]	validation_0-rmse:0.56067	validation_1-rmse:0.59889
[292]	validation_0-rmse:0.55917	validation_1-rmse:0.59740
[293]	validation_0-rmse:0.55771	validation_1-rmse:0.59598
[294]	validation_0-rmse:0.55632	validation_1-rmse:0.59460
[295]	validation_0-rmse:0.55493	validation_1-rmse:0.59328
[296]	validation_0-rmse:0.55349	validation_1-rmse:0.59187
[297]	validation_0-rmse:0.55214	validation_1-rmse:0.59055
[298]	validation_0-rmse:0.55070	validation_1-rmse:0.58918
[299]	validation_0-rmse:0.54973	validation_1-rmse:0.58826
[300]	validation_0-rmse:0.54836	validation_1-rmse:0.58689
[301]	validation_0-rmse:0.54746	validation_1-rmse:0.58609
[302]	validation_0-rmse:0.54614	validation_1-rmse:0.58477
[303]	validati

[428]	validation_0-rmse:0.46748	validation_1-rmse:0.51160
[429]	validation_0-rmse:0.46719	validation_1-rmse:0.51132
[430]	validation_0-rmse:0.46687	validation_1-rmse:0.51102
[431]	validation_0-rmse:0.46656	validation_1-rmse:0.51070
[432]	validation_0-rmse:0.46624	validation_1-rmse:0.51041
[433]	validation_0-rmse:0.46602	validation_1-rmse:0.51027
[434]	validation_0-rmse:0.46556	validation_1-rmse:0.50984
[435]	validation_0-rmse:0.46528	validation_1-rmse:0.50956
[436]	validation_0-rmse:0.46497	validation_1-rmse:0.50926
[437]	validation_0-rmse:0.46466	validation_1-rmse:0.50894
[438]	validation_0-rmse:0.46433	validation_1-rmse:0.50861
[439]	validation_0-rmse:0.46415	validation_1-rmse:0.50848
[440]	validation_0-rmse:0.46382	validation_1-rmse:0.50817
[441]	validation_0-rmse:0.46362	validation_1-rmse:0.50805
[442]	validation_0-rmse:0.46330	validation_1-rmse:0.50776
[443]	validation_0-rmse:0.46304	validation_1-rmse:0.50751
[444]	validation_0-rmse:0.46264	validation_1-rmse:0.50714
[445]	validati

[570]	validation_0-rmse:0.43757	validation_1-rmse:0.48748
[571]	validation_0-rmse:0.43744	validation_1-rmse:0.48743
[572]	validation_0-rmse:0.43723	validation_1-rmse:0.48723
[573]	validation_0-rmse:0.43698	validation_1-rmse:0.48701
[574]	validation_0-rmse:0.43684	validation_1-rmse:0.48690
[575]	validation_0-rmse:0.43672	validation_1-rmse:0.48684
[576]	validation_0-rmse:0.43666	validation_1-rmse:0.48683
[577]	validation_0-rmse:0.43653	validation_1-rmse:0.48674
[578]	validation_0-rmse:0.43644	validation_1-rmse:0.48669
[579]	validation_0-rmse:0.43632	validation_1-rmse:0.48664
[580]	validation_0-rmse:0.43609	validation_1-rmse:0.48642
[581]	validation_0-rmse:0.43598	validation_1-rmse:0.48635
[582]	validation_0-rmse:0.43585	validation_1-rmse:0.48631
[583]	validation_0-rmse:0.43575	validation_1-rmse:0.48627
[584]	validation_0-rmse:0.43567	validation_1-rmse:0.48625
[585]	validation_0-rmse:0.43560	validation_1-rmse:0.48623
[586]	validation_0-rmse:0.43552	validation_1-rmse:0.48620
[587]	validati

...

[19482]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19483]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19484]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19485]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19486]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19487]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19488]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19489]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19490]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19491]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19492]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19493]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19494]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19495]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19496]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19497]	validation_0-rmse:0.32530	validation_1-rmse:0.43466
[19498]	validation_0-rmse:0.32529	valida

[19619]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19620]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19621]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19622]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19623]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19624]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19625]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19626]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19627]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19628]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19629]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19630]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19631]	validation_0-rmse:0.32521	validation_1-rmse:0.43465
[19632]	validation_0-rmse:0.32520	validation_1-rmse:0.43465
[19633]	validation_0-rmse:0.32520	validation_1-rmse:0.43465
[19634]	validation_0-rmse:0.32520	validation_1-rmse:0.43465
[19635]	validation_0-rmse:0.32520	valida

[19756]	validation_0-rmse:0.32513	validation_1-rmse:0.43466
[19757]	validation_0-rmse:0.32513	validation_1-rmse:0.43466
[19758]	validation_0-rmse:0.32513	validation_1-rmse:0.43466
[19759]	validation_0-rmse:0.32513	validation_1-rmse:0.43466
[19760]	validation_0-rmse:0.32513	validation_1-rmse:0.43466
[19761]	validation_0-rmse:0.32513	validation_1-rmse:0.43466
[19762]	validation_0-rmse:0.32513	validation_1-rmse:0.43466
[19763]	validation_0-rmse:0.32512	validation_1-rmse:0.43466
[19764]	validation_0-rmse:0.32512	validation_1-rmse:0.43466
[19765]	validation_0-rmse:0.32512	validation_1-rmse:0.43466
[19766]	validation_0-rmse:0.32512	validation_1-rmse:0.43466
[19767]	validation_0-rmse:0.32512	validation_1-rmse:0.43466
[19768]	validation_0-rmse:0.32512	validation_1-rmse:0.43466
[19769]	validation_0-rmse:0.32512	validation_1-rmse:0.43466
[19770]	validation_0-rmse:0.32512	validation_1-rmse:0.43466
[19771]	validation_0-rmse:0.32512	validation_1-rmse:0.43466
[19772]	validation_0-rmse:0.32512	valida

[19893]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19894]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19895]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19896]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19897]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19898]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19899]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19900]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19901]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19902]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19903]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19904]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19905]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19906]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19907]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19908]	validation_0-rmse:0.32504	validation_1-rmse:0.43465
[19909]	validation_0-rmse:0.32503	valida

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=11, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=20000, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=10, reg_lambda=0.2, scale_pos_weight=1, subsample=0.99,
             tree_method='exact', validate_parameters=1, verbosity=None)

## Generating Predictions

In [82]:
#Predict test set
y_pre_xgbr = xgbr.predict(X_test)

In [83]:
#Sales must >= 0
y_pre_xgbr = np.expm1(y_pre_xgbr)
for i in range(len(y_pre_xgbr)):
    if y_pre_xgbr[i] < 0:
        y_pre_xgbr[i] =0

In [84]:
df_describe = pd.DataFrame(y_pre_xgbr)
df_describe.describe()

,0
count,28512.000000
mean,453.829590
std,1252.458618
min,0.000000
25%,3.698366
50%,27.989964
75%,258.197212
max,16295.515625


In [85]:
#Create the outcome file for the competition
outcome = pd.DataFrame()
outcome["id"] = df_test.iloc[:,0]
outcome["sales"] = y_pre_xgbr
outcome

,id,sales
0,3000888,3.557061
1,3000889,0.079824
2,3000890,5.345271
3,3000891,2425.907471
4,3000892,0.497149
...,...,...
28507,3029395,342.231506
28508,3029396,96.564987
28509,3029397,1331.631714
28510,3029398,43.546394


In [86]:
outcome.to_csv('Sales_predict.csv', index=False)